In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [13]:
#构建模型前向传播
class soft_net(nn.Module):

    def __init__(self,num1,num2,num3,num4):
        super(soft_net,self).__init__()
        self.net1 = nn.Sequential(nn.Linear(num1,num2),
                                    nn.Sigmoid(),
                                    nn.Linear(num2,num3),
                                    nn.Sigmoid(),
                                    nn.Linear(num3,num4),
                                    nn.Sigmoid())
        
        self.net2 = nn.Sequential(nn.Linear(num1,num2),
                                    nn.Sigmoid(),
                                    nn.Linear(num2,num3),
                                    nn.Sigmoid(),
                                    nn.Linear(num3,num4),
                                    nn.Sigmoid())
        
        self.net3 = nn.Sequential(nn.Linear(num1,num2),
                                    nn.Sigmoid(),
                                    nn.Linear(num2,num3),
                                    nn.Sigmoid(),
                                    nn.Linear(num3,num4),
                                    nn.Sigmoid())
    
    def forward(self,x):

        y1 = self.net1(x)
        y2 = self.net2(x)
        y3 = self.net3(x)

        return torch.cat([y1,y2,y3],1)

#定义损失函数
class loss_funcation():
    def __init__(self,model,punish=0.01):
        self.punish = punish
        self.model = model
    
    def __call__(self,y_hat,y,index=None):
        loss1 = (torch.sum((y-y_hat)**2)/(3*y.shape[0]))
        #读取参数
        param_list = []
        for name,parameter in self.model.named_parameters():
            param_list += list(parameter.flatten())  
        param = torch.tensor(param_list).reshape(3,-1)
        avg_param = torch.sum(param,dim=0)/3
        if index is None:
            loss2 = (((torch.sum((param[0]-avg_param)**2))**0.5)+((torch.sum((param[1]-avg_param)**2))**0.5)+((torch.sum((param[2]-avg_param)**2))**0.5))*self.punish
        else:
            loss2 = (((torch.sum((param[0][index]-avg_param[index])**2))**0.5)+((torch.sum((param[1][index]-avg_param[index])**2))**0.5)+((torch.sum((param[2][index]-avg_param[index])**2))**0.5))*self.punish
        return loss1+loss2

#导出参数序号
def param_index(model,rate):
    #读取参数
    param_list = []
    for name,parameter in model.named_parameters():
        param_list += list(parameter.flatten())  
    param = torch.tensor(param_list).reshape(3,-1)
    avg_param = torch.sum(param,dim=0)/3
    difference = abs(param[0]-avg_param)+abs(param[1]-avg_param)+abs(param[2]-avg_param)
    index = torch.argsort(difference)[:int(difference.shape[0]*(1-rate))]
    return index

#归一化
def data_one(data):
    for i in range(data.shape[1]):
        ma = torch.max(data[:,i])
        mi = torch.min(data[:,i])
        cha = ma-mi
        # print(ma,mi,min(data[:,i]))
        for k in range(data.shape[0]):
            data[k,i] = (data[k,i]-mi)/cha
    return data

#创建模拟数据
def gen_data(w11,w12,w21,w22,w31,w32,m=1000,n=5,require=None,x_type="u"):
    if x_type == "n":
        x = torch.normal(0,1,(m,n))
    if x_type == "e":
        x = torch.tensor(np.random.exponential(10,(m,n)),dtype=torch.float32)
    if x_type == "u":
        x = torch.tensor(np.random.uniform(0,1,(m,n)),dtype=torch.float32)

    if require == None:
        noise = 0
        
    else:
        noise = torch.normal(0,1,(m,3))
    y1 = x@w11@w12
    # y1 = np.sin(x)@w11@w12
    y2 = x@w21@w22
    y3 = x@w31@w32
    
    label = torch.cat([y1,y2,y3],1)+noise
    return x,label

class gen_data_set(torch.utils.data.Dataset):
    #封装dataset
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self,index):
        x = self.x[index]
        y = self.y[index]
        return x,y
#训练模型
def train_model(model,loader_dict,optim,epoches,loss_f,index=None):
    for epoch in range(epoches):
        # print(f"epoch {epoch+1} / {epoches}")
        # print("-"*100)
        for phase in ["train","eval"]:
            epoch_loss = 0
            for input,label in loader_dict[phase]:
                optim.zero_grad()
                with torch.set_grad_enabled(phase=="train"):
                    output = model(input)
                    loss = loss_f(output,label,index)*input.shape[0]
                    if phase == "train":
                        loss.backward()
                        optim.step()
                    epoch_loss += loss
            print(f"{epoch}  {phase} : {epoch_loss/len(loader_dict[phase].dataset)}")

In [6]:
data = torch.rand(100,3)


tensor([0.1836, 0.8546, 0.6698, 0.5148, 0.6919, 0.6560, 0.3391, 0.3830, 0.2702,
        0.6643])

In [4]:
df = pd.read_excel(r"C:\Users\admin\Desktop\软参数论文\实例数据\2014-2022年11月空气污染数据(均值填充).xlsx")
from sklearn.model_selection import train_test_split
def data_one(data):
    for i in range(data.shape[1]):
        ma = torch.max(data[:,i])
        mi = torch.min(data[:,i])
        cha = ma-mi
        for k in range(data.shape[0]):
            data[k,i] = (data[k,i]-mi)/cha
    return data
x = df.loc[:,("SO2","CO","NO2","PM10")].values
y = df.loc[:,("PM2.5","O3_8h")].values
x = torch.tensor(x,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.float32)
x = data_one(x)
y = data_one(y)
x_train,x_test,y_train,y_test = train_test_split(x,y)

input,label = gen_data(w11,w12,w21,w22,m=m,n=n,require=noise_require)
input1 = data_one(input.clone())
label1 = data_one(label.clone())
train_x,test_x,train_y,test_y = train_test_split(input1,label1,train_size=0.8)
train_dataset = gen_data_set(x_train,y_train)
eval_dataset = gen_data_set(x_test,y_test)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=min_batch,shuffle=True)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset,batch_size=min_batch,shuffle=False)
dataloader_dict = {"train":train_dataloader,
                    "eval":eval_dataloader}

In [11]:
#各类超参数
m = 2000#样本量
n = 4#特征维度
min_batch = 64#小批量
w11 = torch.randn(n,3)#第一个任务第一层权重
# w12 = torch.randn(3,1)#第一个任务第二层权重
w21 = torch.normal(0,1,(n,3))
w31 = torch.normal(0,1,(n,3))
# w22 = torch.randn(3,1)
w12 = torch.tensor([[1.0],[2.0],[3.0]])
w22 = torch.tensor([[1.15],[2.15],[3.15]])
w32 = torch.tensor([[1.45],[2.45],[3.45]])
punish = 0.1#惩罚项
lr = 0.001#学习率
noise_require = True #True

In [14]:
#模拟
input,label = gen_data(w11,w12,w21,w22,w31,w32,m=m,n=n,require=noise_require)
input1 = data_one(input.clone())
label1 = data_one(label.clone())
train_x,test_x,train_y,test_y = train_test_split(input1,label1,train_size=0.8)
train_dataset = gen_data_set(train_x,train_y)
eval_dataset = gen_data_set(test_x,test_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=min_batch,shuffle=True)
eval_dataloader = torch.utils.data.DataLoader(eval_dataset,batch_size=min_batch,shuffle=False)
dataloader_dict = {"train":train_dataloader,
                    "eval":eval_dataloader}

In [15]:
#软参数
model = soft_net(n,4,3,1)
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss_f = loss_funcation(model,punish)
train_model(model,dataloader_dict,optim,1510,loss_f)

0  train : 0.4726378619670868
0  eval : 0.47119712829589844
1  train : 0.4679509401321411
1  eval : 0.46744465827941895
2  train : 0.46494337916374207
2  eval : 0.4651956260204315
3  train : 0.4637190103530884
3  eval : 0.4643253982067108
4  train : 0.46278801560401917
4  eval : 0.4637557566165924
5  train : 0.4631703495979309
5  eval : 0.4647800326347351
6  train : 0.4638335704803467
6  eval : 0.4656916558742523
7  train : 0.4662303626537323
7  eval : 0.46923720836639404
8  train : 0.470556378364563
8  eval : 0.4746793806552887
9  train : 0.4768320918083191
9  eval : 0.48203736543655396
10  train : 0.48523035645484924
10  eval : 0.49117302894592285
11  train : 0.49500176310539246
11  eval : 0.5019215941429138
12  train : 0.506671130657196
12  eval : 0.514330267906189
13  train : 0.5197524428367615
13  eval : 0.5280465483665466
14  train : 0.5341962575912476
14  eval : 0.5432438850402832
15  train : 0.5499667525291443
15  eval : 0.5596950650215149
16  train : 0.5667027235031128
16  eva

In [58]:
#不加惩罚项
model = soft_net(n,4,3,1)
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss_f = loss_funcation(model,punish=0)
train_model(model,dataloader_dict,optim,1510,loss_f)

0  train : 0.03949737921357155
0  eval : 0.036017198115587234
1  train : 0.036090537905693054
1  eval : 0.03308096155524254
2  train : 0.03343590348958969
2  eval : 0.030868573114275932
3  train : 0.031427547335624695
3  eval : 0.02919783629477024
4  train : 0.02992262877523899
4  eval : 0.0279422327876091
5  train : 0.028788071125745773
5  eval : 0.027052784338593483
6  train : 0.02797422930598259
6  eval : 0.026379607617855072
7  train : 0.02736586518585682
7  eval : 0.025936029851436615
8  train : 0.02694414183497429
8  eval : 0.02562357857823372
9  train : 0.026653625071048737
9  eval : 0.02541288547217846
10  train : 0.026452917605638504
10  eval : 0.025270869955420494
11  train : 0.02631966769695282
11  eval : 0.025183530524373055
12  train : 0.02620605193078518
12  eval : 0.025113685056567192
13  train : 0.026129603385925293
13  eval : 0.025059273466467857
14  train : 0.026059361174702644
14  eval : 0.025006122887134552
15  train : 0.02599252201616764
15  eval : 0.02495585940778

KeyboardInterrupt: 

In [7]:
model = soft_net(n,4,3,1)
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss_f = loss_funcation(model,punish=punish)

In [8]:
# for _ in range(100):
train_model(model,dataloader_dict,optim,1500,loss_f)

0  train : 0.0626058503985405
0  eval : 0.058041058480739594
1  train : 0.054535701870918274
1  eval : 0.05087447166442871
2  train : 0.047879595309495926
2  eval : 0.04490097984671593
3  train : 0.042352382093667984
3  eval : 0.040035899728536606
4  train : 0.03786049038171768
4  eval : 0.03610847890377045
5  train : 0.034276559948921204
5  eval : 0.03297870233654976
6  train : 0.03142096847295761
6  eval : 0.030532842501997948
7  train : 0.029182635247707367
7  eval : 0.028629304841160774
8  train : 0.027429861947894096
8  eval : 0.027132989838719368
9  train : 0.026051629334688187
9  eval : 0.025968197733163834
10  train : 0.024974804371595383
10  eval : 0.025044146925210953
11  train : 0.024124037474393845
11  eval : 0.024320490658283234
12  train : 0.023445844650268555
12  eval : 0.023748768493533134
13  train : 0.02291049063205719
13  eval : 0.023290587589144707
14  train : 0.022477535530924797
14  eval : 0.02294267527759075
15  train : 0.022135093808174133
15  eval : 0.022652247

In [18]:
index = param_index(model,0.04)
loss_f = loss_funcation(model,punish=0.0007)

In [19]:
# for _ in range(20):
train_model(model,dataloader_dict,optim,30,loss_f,index)

0  train : 0.026228101924061775
0  eval : 0.027669798582792282
1  train : 0.026228783652186394
1  eval : 0.02766379714012146
2  train : 0.02623889222741127
2  eval : 0.027666205540299416
3  train : 0.02624153345823288
3  eval : 0.02767244353890419
4  train : 0.026237191632390022
4  eval : 0.027696838602423668
5  train : 0.026240959763526917
5  eval : 0.02768179029226303
6  train : 0.026239439845085144
6  eval : 0.02768845669925213
7  train : 0.026240069419145584
7  eval : 0.027702324092388153
8  train : 0.026256730780005455
8  eval : 0.0276879221200943
9  train : 0.026246052235364914
9  eval : 0.027710435912013054
10  train : 0.02625187113881111
10  eval : 0.027701660990715027
11  train : 0.02625846117734909
11  eval : 0.027712292969226837
12  train : 0.026255832985043526
12  eval : 0.02771107852458954
13  train : 0.026259947568178177
13  eval : 0.027703486382961273
14  train : 0.0262630395591259
14  eval : 0.027713505551218987
15  train : 0.026268787682056427
15  eval : 0.027720995247